In [1]:
import pandas as pd
import numpy as np

In [2]:
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 

Using TensorFlow backend.


In [3]:
import tensorflow as tf
import random as rn
from keras import backend as K

In [4]:
from keras.models import load_model
model = load_model('models/train_no_null.h5')

# Load Train, Valid and Test data

In [5]:
import pickle

In [6]:
global x_train, x_test, x_valid, y_train, y_test, y_valid

In [7]:
with open('x_train.pkl', 'rb') as fp:
    x_train = pickle.load(fp)
with open('x_test.pkl', 'rb') as fp:
    x_test = pickle.load(fp)
with open('x_valid.pkl', 'rb') as fp:
    x_valid = pickle.load(fp)
with open('y_train.pkl', 'rb') as fp:
    y_train = pickle.load(fp)
with open('y_test.pkl', 'rb') as fp:
    y_test = pickle.load(fp)
with open('y_valid.pkl', 'rb') as fp:
    y_valid = pickle.load(fp)

# Load Best Params

In [8]:
global best_params

In [8]:
with open('best_params.pkl', 'rb') as fp:
    best_params = pickle.load(fp)

In [9]:
best_params

{'activation': 'relu',
 'batch_size': 32,
 'dropout1': 0.36373495616497264,
 'dropout2': 0.6121737368795496,
 'early_stop_rounds': 40,
 'nb_epochs': 500,
 'num_layers': 'one_hidden',
 'optimizer': 'nadam',
 'units1': 256,
 'units2': 128}

# Results on all data

### Test data

In [10]:
loss,mse = model.evaluate(x_test,y_test,verbose=0)

In [11]:
rmse = loss ** 0.5
rmse

34985.13477075421

In [12]:
pct_error = (loss ** 0.5) / y_test.mean()*100
pct_error

19.437331888312688

### Valid data

In [13]:
loss_v,mse_v = model.evaluate(x_valid,y_valid,verbose=0)

In [14]:
rmse_v = loss_v ** 0.5
rmse_v

40898.65497425861

In [15]:
pct_error_v = (loss_v ** 0.5) / y_valid.mean()*100
pct_error_v

21.85238068805628

# Intensity DF 

In [16]:
df = pd.read_csv('house_pred_intensity_df.csv')

In [20]:
df.reset_index(inplace=True)

In [52]:
def train_best_model(best_params,feature_subset):   
#     print('Training the best selected model...') 

    import os
    os.environ['PYTHONHASHSEED'] = '0'

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.

    np.random.seed(42)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.

    rn.seed(12345)

    # Force TensorFlow to use single thread.
    # Multiple threads are a potential source of
    # non-reproducible results.
    # For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    # The below tf.set_random_seed() will make random number generation
    # in the TensorFlow backend have a well-defined initial state.
    # For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

    tf.set_random_seed(1234)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)

    # ---------------------------------------------------
    
    x_train_temp = x_train.copy() 
    y_train_temp = y_train.copy()
    
    x_valid_temp = x_valid.copy() 
    y_valid_temp = y_valid.copy()
    
    x_train_temp = x_train_temp[feature_subset]
    x_valid_temp = x_valid_temp[feature_subset]
    
    
    
    model = Sequential()
    model.add(Dense(best_params['units1'], input_shape=(x_train_temp.shape[1],)))
    model.add(Activation(best_params['activation']))
    model.add(Dropout(best_params['dropout1']))
    if(best_params['num_layers'] == 'two_hidden'):
        model.add(Dense(best_params['units2']))
        model.add(Activation(best_params['activation']))
        model.add(Dropout(best_params['dropout2']))
    model.add(Dense(1))
    model.add(Activation('linear'))
    model.compile(loss='mse', metrics=['mse'],
                  optimizer=best_params['optimizer'])
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=best_params['early_stop_rounds'])
    history = History()
    model.fit(x_train_temp, y_train_temp,
              batch_size=best_params['batch_size'],
              epochs=500,
              callbacks=[early_stop, history],
              verbose=0,
              validation_data=(x_valid_temp,y_valid_temp)) 

    return model

In [21]:
def get_results(best_params,feature_subset):
    model = train_best_model(best_params,feature_subset)
    x_valid_temp = x_valid.copy() 
    y_valid_temp = y_valid.copy()
    
    x_test_temp = x_test.copy() 
    y_test_temp = y_test.copy()
    
    x_valid_temp = x_valid_temp[feature_subset]
    x_test_temp = x_test_temp[feature_subset]
    
    loss,mse = model.evaluate(x_test_temp,y_test_temp,verbose=0)
    rmse = loss ** 0.5
    pct_error = (loss ** 0.5) / y_test_temp.mean()*100
    
    loss_v,mse_v = model.evaluate(x_valid_temp,y_valid_temp,verbose=0)
    rmse_v = loss_v ** 0.5
    pct_error_v = (loss_v ** 0.5) / y_valid_temp.mean()*100
    
    return rmse, pct_error, rmse_v, pct_error_v

# Testing

In [15]:
df1 = pd.read_csv('house_pred_intensity_df.csv')

In [28]:
f = list(df['feature_name'])

In [17]:
f1 = list(df1['feature_name'])

In [29]:
rmse, pct_error, rmse_v, pct_error_v = get_results(best_params,f)

396
Train on 870 samples, validate on 290 samples
Epoch 1/500
870/870 [==============================] - 0s 408us/step - loss: 34472061339.9540 - mean_squared_error: 34472061339.9540 - val_loss: 32321957902.1241 - val_mean_squared_error: 32321957902.1241
Epoch 2/500
870/870 [==============================] - 0s 110us/step - loss: 23831477245.6460 - mean_squared_error: 23831477245.6460 - val_loss: 18316696653.6828 - val_mean_squared_error: 18316696653.6828
Epoch 3/500
870/870 [==============================] - 0s 109us/step - loss: 13368228395.5494 - mean_squared_error: 13368228395.5494 - val_loss: 9144594791.2828 - val_mean_squared_error: 9144594791.2828
Epoch 4/500
870/870 [==============================] - 0s 108us/step - loss: 8886484043.9172 - mean_squared_error: 8886484043.9172 - val_loss: 6119100989.7931 - val_mean_squared_error: 6119100989.7931
Epoch 5/500
870/870 [==============================] - 0s 111us/step - loss: 7461228948.8920 - mean_squared_error: 7461228948.8920 - val

870/870 [==============================] - 0s 111us/step - loss: 2434914739.7885 - mean_squared_error: 2434914739.7885 - val_loss: 1813343466.8138 - val_mean_squared_error: 1813343466.8138
Epoch 42/500
870/870 [==============================] - 0s 110us/step - loss: 2515486322.1701 - mean_squared_error: 2515486322.1701 - val_loss: 1973698812.0276 - val_mean_squared_error: 1973698812.0276
Epoch 43/500
870/870 [==============================] - 0s 110us/step - loss: 2475597919.3379 - mean_squared_error: 2475597919.3379 - val_loss: 1880072031.4483 - val_mean_squared_error: 1880072031.4483
Epoch 44/500
870/870 [==============================] - 0s 110us/step - loss: 2372018042.7034 - mean_squared_error: 2372018042.7034 - val_loss: 1835924977.6552 - val_mean_squared_error: 1835924977.6552
Epoch 45/500
870/870 [==============================] - 0s 110us/step - loss: 2731842361.0851 - mean_squared_error: 2731842361.0851 - val_loss: 1755115396.4138 - val_mean_squared_error: 1755115396.4138
Epo

Epoch 82/500
870/870 [==============================] - 0s 110us/step - loss: 2415518636.4322 - mean_squared_error: 2415518636.4322 - val_loss: 1706911072.8828 - val_mean_squared_error: 1706911072.8828
Epoch 83/500
870/870 [==============================] - 0s 109us/step - loss: 2220229391.8897 - mean_squared_error: 2220229391.8897 - val_loss: 1673094889.4345 - val_mean_squared_error: 1673094889.4345
Epoch 84/500
870/870 [==============================] - 0s 108us/step - loss: 2300158875.9540 - mean_squared_error: 2300158875.9540 - val_loss: 1711024810.5931 - val_mean_squared_error: 1711024810.5931
Epoch 85/500
870/870 [==============================] - 0s 109us/step - loss: 2154958436.1195 - mean_squared_error: 2154958436.1195 - val_loss: 1660410486.1793 - val_mean_squared_error: 1660410486.1793
Epoch 86/500
870/870 [==============================] - 0s 109us/step - loss: 2175986005.3333 - mean_squared_error: 2175986005.3333 - val_loss: 1663534310.5103 - val_mean_squared_error: 166353

870/870 [==============================] - 0s 109us/step - loss: 2183830669.5356 - mean_squared_error: 2183830669.5356 - val_loss: 1662461474.2069 - val_mean_squared_error: 1662461474.2069
Epoch 123/500
870/870 [==============================] - 0s 111us/step - loss: 2111105442.3540 - mean_squared_error: 2111105442.3540 - val_loss: 1656637069.2414 - val_mean_squared_error: 1656637069.2414
Epoch 124/500
870/870 [==============================] - 0s 109us/step - loss: 2030093390.7862 - mean_squared_error: 2030093390.7862 - val_loss: 1730581440.9931 - val_mean_squared_error: 1730581440.9931
Epoch 125/500
870/870 [==============================] - 0s 111us/step - loss: 2258834569.1218 - mean_squared_error: 2258834569.1218 - val_loss: 1779868774.0690 - val_mean_squared_error: 1779868774.0690
Epoch 126/500
870/870 [==============================] - 0s 111us/step - loss: 2196983913.3425 - mean_squared_error: 2196983913.3425 - val_loss: 1717836696.7448 - val_mean_squared_error: 1717836696.7448

In [30]:
rmse, pct_error, rmse_v, pct_error_v

(35675.11526823791, 19.820676986564678, 41424.94310802911, 22.13357938904041)

In [21]:
rmse, pct_error, rmse_v, pct_error_v = get_results(best_params,f1)

396
Train on 870 samples, validate on 290 samples
Epoch 1/500
870/870 [==============================] - 1s 645us/step - loss: 17239474700.3586 - mean_squared_error: 17239474700.3586 - val_loss: 5220950585.3793 - val_mean_squared_error: 5220950585.3793
Epoch 2/500
870/870 [==============================] - 0s 263us/step - loss: 7097127079.4299 - mean_squared_error: 7097127079.4299 - val_loss: 4091925707.9172 - val_mean_squared_error: 4091925707.9172
Epoch 3/500
870/870 [==============================] - 0s 259us/step - loss: 4779007841.1034 - mean_squared_error: 4779007841.1034 - val_loss: 3060018774.5103 - val_mean_squared_error: 3060018774.5103
Epoch 4/500
870/870 [==============================] - 0s 261us/step - loss: 3892789036.1379 - mean_squared_error: 3892789036.1379 - val_loss: 2619949918.0138 - val_mean_squared_error: 2619949918.0138
Epoch 5/500
870/870 [==============================] - 0s 264us/step - loss: 3609646170.7034 - mean_squared_error: 3609646170.7034 - val_loss: 2

Epoch 82/500
870/870 [==============================] - 0s 261us/step - loss: 1774307539.2736 - mean_squared_error: 1774307539.2736 - val_loss: 2243614284.3586 - val_mean_squared_error: 2243614284.3586
Epoch 83/500
870/870 [==============================] - 0s 266us/step - loss: 1825978157.1678 - mean_squared_error: 1825978157.1678 - val_loss: 1591641062.1793 - val_mean_squared_error: 1591641062.1793
Epoch 84/500
870/870 [==============================] - 0s 266us/step - loss: 1788730721.6920 - mean_squared_error: 1788730721.6920 - val_loss: 1569999120.5517 - val_mean_squared_error: 1569999120.5517
Epoch 85/500
870/870 [==============================] - 0s 260us/step - loss: 1752451546.9241 - mean_squared_error: 1752451546.9241 - val_loss: 1648279241.7103 - val_mean_squared_error: 1648279241.7103
Epoch 86/500
870/870 [==============================] - 0s 260us/step - loss: 1699597335.5402 - mean_squared_error: 1699597335.5402 - val_loss: 1493729471.5586 - val_mean_squared_error: 149372

870/870 [==============================] - 0s 261us/step - loss: 1158225668.3402 - mean_squared_error: 1158225668.3402 - val_loss: 1118620584.4414 - val_mean_squared_error: 1118620584.4414
Epoch 163/500
870/870 [==============================] - 0s 262us/step - loss: 1853630367.4851 - mean_squared_error: 1853630367.4851 - val_loss: 2038064165.5724 - val_mean_squared_error: 2038064165.5724
Epoch 164/500
870/870 [==============================] - 0s 260us/step - loss: 1965805031.2828 - mean_squared_error: 1965805031.2828 - val_loss: 1623762220.4690 - val_mean_squared_error: 1623762220.4690
Epoch 165/500
870/870 [==============================] - 0s 262us/step - loss: 1315129710.0506 - mean_squared_error: 1315129710.0506 - val_loss: 1577603031.3931 - val_mean_squared_error: 1577603031.3931
Epoch 166/500
870/870 [==============================] - 0s 264us/step - loss: 1318246385.1402 - mean_squared_error: 1318246385.1402 - val_loss: 1199016736.1103 - val_mean_squared_error: 1199016736.1103

In [22]:
rmse, pct_error, rmse_v, pct_error_v

(34433.73763803584, 19.13098208742896, 39876.03827449469, 21.305990851146735)

# Dropping Zero Intensities

In [31]:
df_non_zero = df[df['sum_of_intensities'] != 0]

In [32]:
non_zero_features = list(df_non_zero['feature_name'])

In [33]:
rmse, pct_error, rmse_v, pct_error_v = get_results(best_params,non_zero_features)

280
Train on 870 samples, validate on 290 samples
Epoch 1/500
870/870 [==============================] - 0s 433us/step - loss: 35574007812.7080 - mean_squared_error: 35574007812.7080 - val_loss: 34774573889.3241 - val_mean_squared_error: 34774573889.3241
Epoch 2/500
870/870 [==============================] - 0s 112us/step - loss: 27417852534.8782 - mean_squared_error: 27417852534.8782 - val_loss: 23517212926.2345 - val_mean_squared_error: 23517212926.2345
Epoch 3/500
870/870 [==============================] - 0s 108us/step - loss: 18283679160.2023 - mean_squared_error: 18283679160.2023 - val_loss: 14392677594.9241 - val_mean_squared_error: 14392677594.9241
Epoch 4/500
870/870 [==============================] - 0s 107us/step - loss: 13306831215.2276 - mean_squared_error: 13306831215.2276 - val_loss: 9735694607.8897 - val_mean_squared_error: 9735694607.8897
Epoch 5/500
870/870 [==============================] - 0s 108us/step - loss: 11740242468.4874 - mean_squared_error: 11740242468.4874

870/870 [==============================] - 0s 108us/step - loss: 2394700209.1402 - mean_squared_error: 2394700209.1402 - val_loss: 1802282214.1793 - val_mean_squared_error: 1802282214.1793
Epoch 42/500
870/870 [==============================] - 0s 108us/step - loss: 2717880418.8690 - mean_squared_error: 2717880418.8690 - val_loss: 1929458729.4897 - val_mean_squared_error: 1929458729.4897
Epoch 43/500
870/870 [==============================] - 0s 107us/step - loss: 2411508309.9218 - mean_squared_error: 2411508309.9218 - val_loss: 1873219901.5724 - val_mean_squared_error: 1873219901.5724
Epoch 44/500
870/870 [==============================] - 0s 107us/step - loss: 2465171111.1356 - mean_squared_error: 2465171111.1356 - val_loss: 1791374696.3862 - val_mean_squared_error: 1791374696.3862
Epoch 45/500
870/870 [==============================] - 0s 107us/step - loss: 2759936700.3218 - mean_squared_error: 2759936700.3218 - val_loss: 1715894073.8207 - val_mean_squared_error: 1715894073.8207
Epo

Epoch 82/500
870/870 [==============================] - 0s 110us/step - loss: 2432963634.6115 - mean_squared_error: 2432963634.6115 - val_loss: 1724993866.9241 - val_mean_squared_error: 1724993866.9241
Epoch 83/500
870/870 [==============================] - 0s 107us/step - loss: 2380246781.9402 - mean_squared_error: 2380246781.9402 - val_loss: 1695086890.6483 - val_mean_squared_error: 1695086890.6483
Epoch 84/500
870/870 [==============================] - 0s 107us/step - loss: 2269452378.6299 - mean_squared_error: 2269452378.6299 - val_loss: 1753948430.4552 - val_mean_squared_error: 1753948430.4552
Epoch 85/500
870/870 [==============================] - 0s 108us/step - loss: 2323825220.3402 - mean_squared_error: 2323825220.3402 - val_loss: 1747460706.3172 - val_mean_squared_error: 1747460706.3172
Epoch 86/500
870/870 [==============================] - 0s 107us/step - loss: 2278804019.7885 - mean_squared_error: 2278804019.7885 - val_loss: 1742724884.8000 - val_mean_squared_error: 174272

870/870 [==============================] - 0s 106us/step - loss: 2205106213.8851 - mean_squared_error: 2205106213.8851 - val_loss: 1738783467.6414 - val_mean_squared_error: 1738783467.6414
Epoch 123/500
870/870 [==============================] - 0s 110us/step - loss: 2135124189.8667 - mean_squared_error: 2135124189.8667 - val_loss: 1743373080.9655 - val_mean_squared_error: 1743373080.9655
Epoch 124/500
870/870 [==============================] - 0s 108us/step - loss: 2200435534.2713 - mean_squared_error: 2200435534.2713 - val_loss: 1799393179.2276 - val_mean_squared_error: 1799393179.2276
Epoch 125/500
870/870 [==============================] - 0s 108us/step - loss: 2294152221.4253 - mean_squared_error: 2294152221.4253 - val_loss: 1820382365.4897 - val_mean_squared_error: 1820382365.4897
Epoch 126/500
870/870 [==============================] - 0s 108us/step - loss: 2315855894.9517 - mean_squared_error: 2315855894.9517 - val_loss: 1788451768.7172 - val_mean_squared_error: 1788451768.7172

In [34]:
rmse, pct_error, rmse_v, pct_error_v

(36565.48059412098, 20.31535355289599, 41356.5480250918, 22.097035512708093)

In [38]:
df_non_zero_sorted = df_non_zero.sort_values(by=['sum_of_intensities'],ascending=False)

# Simulation

In [48]:
def simulation_of_features(df):
    feature_dict = {}
    if(len(df)<=100):
        for i in range(1,100):
            drop_value = i
            print(i,'No of features dropped:',drop_value)
            subset_df = df.head(len(df) - drop_value)
            subset_df.sort_values(by=['index'],inplace=True)
            feature_subset = list(subset_df['feature_name'])

            rmse, pct_error, rmse_v, pct_error_v = get_results(best_params,feature_subset)

            feature_dict[drop_value] = rmse, pct_error, rmse_v, pct_error_v
        return feature_dict
        
    else:
        for i in range(1,100):
            drop_value = int(np.floor((i / 100) * len(df)))
            print(i,'No of features dropped:',drop_value)
            subset_df = df.head(len(df) - drop_value)
            subset_df.sort_values(by=['index'],inplace=True)
            feature_subset = list(subset_df['feature_name'])
            
            rmse, pct_error, rmse_v, pct_error_v = get_results(best_params,feature_subset)
            
            feature_dict[drop_value] = rmse, pct_error, rmse_v, pct_error_v
        return feature_dict

In [53]:
resultant_dict = simulation_of_features(df_non_zero_sorted)

1 No of features dropped: 2


/home/ubuntu/izenda/env/lib/python3.5/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2 No of features dropped: 5
3 No of features dropped: 8
4 No of features dropped: 11
5 No of features dropped: 14
6 No of features dropped: 16
7 No of features dropped: 19
8 No of features dropped: 22
9 No of features dropped: 25
10 No of features dropped: 28
11 No of features dropped: 30
12 No of features dropped: 33
13 No of features dropped: 36
14 No of features dropped: 39
15 No of features dropped: 42
16 No of features dropped: 44
17 No of features dropped: 47
18 No of features dropped: 50
19 No of features dropped: 53
20 No of features dropped: 56
21 No of features dropped: 58
22 No of features dropped: 61
23 No of features dropped: 64
24 No of features dropped: 67
25 No of features dropped: 70
26 No of features dropped: 72
27 No of features dropped: 75
28 No of features dropped: 78
29 No of features dropped: 81
30 No of features dropped: 84
31 No of features dropped: 86
32 No of features dropped: 89
33 No of features dropped: 92
34 No of features dropped: 95
35 No of features dr

In [54]:
resultant_dict

{2: (36495.71739541758,
  20.276593935256276,
  41284.968443649115,
  22.058790140967588),
 5: (36637.26862071263,
  20.35523814123936,
  41306.33148461917,
  22.070204533549102),
 8: (36632.22250322485,
  20.3524345773543,
  41342.38864126645,
  22.089470074532816),
 11: (36605.178669548295,
  20.337409339516913,
  41361.593637009486,
  22.099731411448502),
 14: (36572.69901794294,
  20.319364024785187,
  41288.52589379716,
  22.06069090652975),
 16: (36569.44401021835,
  20.317555580545758,
  41340.483904212866,
  22.088452362844453),
 19: (36641.85444668564,
  20.357785972539812,
  41434.259709602135,
  22.138557301502832),
 22: (36540.19126642527,
  20.30130309807068,
  41401.16203899729,
  22.12087303050663),
 25: (36372.24257315855,
  20.20799276747957,
  41307.491388366834,
  22.07082427662553),
 28: (36360.70337707456,
  20.201581725033158,
  41289.24398502802,
  22.061074586701025),
 30: (36439.309977208955,
  20.245254633125004,
  41304.45539898688,
  22.0692021304758),
 33: 

In [56]:
resultant_df = pd.DataFrame(resultant_dict).T

In [57]:
resultant_df.reset_index(inplace=True)

In [58]:
resultant_df.columns = ['no_of_dropped_features', 'rmse_test', 'pct_error_test', 'rmse_valid', 'pct_error_valid']

In [59]:
temp_df = pd.DataFrame({'no_of_dropped_features':[0],'rmse_test':[36565.48059412098],'pct_error_test':[20.31535355289599],'rmse_valid':[41356.5480250918],'pct_error_valid':[22.097035512708093]})

In [60]:
resultant_df = resultant_df.append(temp_df)

/home/ubuntu/izenda/env/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [61]:
resultant_df.sort_values(by=['no_of_dropped_features'],inplace=True)
resultant_df.reset_index(drop=True,inplace=True)

In [62]:
resultant_df.to_csv('house_pred_lime_feature_selection_results.csv')

In [63]:
a = pd.read_csv('house_pred_lime_feature_selection_results.csv',index_col=0)
a

,no_of_dropped_features,pct_error_test,pct_error_valid,rmse_test,rmse_valid
0,0,20.315354,22.097036,36565.480594,41356.548025
1,2,20.276594,22.058790,36495.717395,41284.968444
2,5,20.355238,22.070205,36637.268621,41306.331485
3,8,20.352435,22.089470,36632.222503,41342.388641
4,11,20.337409,22.099731,36605.178670,41361.593637
5,14,20.319364,22.060691,36572.699018,41288.525894
6,16,20.317556,22.088452,36569.444010,41340.483904
7,19,20.357786,22.138557,36641.854447,41434.259710
8,22,20.301303,22.120873,36540.191266,41401.162039
9,25,20.207993,22.070824,36372.242573,41307.491388
